In [1]:
import requests
import json
import math
import re
import os
import time

Выкачиваем названия корневых коллекций:

In [5]:
auth = 'Token token=avgeck6iayd42klx'

In [8]:
url = 'http://api.repo.nypl.org/api/v1/items/roots'
call = requests.get(url, headers={'Authorization': auth})
uuids_col = json.loads(call.text)['nyplAPI']['response']['uuids']['uuid']
with open('all_cols_uuids.json', 'w') as outfile:
    json.dump(uuids_col, outfile)

Загружаем массив корневых коллекций:

In [2]:
uuids_col = json.loads(open('all_cols_uuids.json').read())

In [22]:
len(uuids_col)

4489

In [3]:
url_col = 'http://api.repo.nypl.org/api/v1/collections/'

Проверяем, совпадает ли количество заявленных элементов с количеством скачанных:

In [20]:
i = 0
for uuid_col in uuids_col:
    i += 1
    try:
        call = requests.get(url_col + uuid_col, headers={'Authorization': auth})
    except:
        time.sleep(5)
        call = requests.get(url_col + uuid_col, headers={'Authorization': auth})
    num_items = json.loads(call.text)['nyplAPI']['response']['numItems']
    items = json.loads(open(os.path.join('./topics', 'items_' + uuid_col + '.json')).read())

    if int(num_items) != len(items):
        print('!!!', uuid_col, num_items, len(items))

!!! d6d29460-031a-0133-7ddc-58d385a7b928 801 567
!!! a1a9d830-c5a6-012f-00ec-58d385a7bc34 1860 1813
!!! 846c47a0-004d-0130-0ebc-58d385a7bc34 9 8
!!! 103776c0-0042-0130-23a4-58d385a7bc34 1 0


In [21]:
i

4489

Оставляем только элементы, у которых есть топики, и записываем их в файл вида "topics_COLLECTION-UUID"; смотрим процент элементов, у которых нет топиков:

In [3]:
with_topics = 0
all_items = 0
for root, dirs, files in os.walk('./topics'):
    for file in files:
        if file.startswith('items_'):
            topics = {}
            items = json.loads(open(os.path.join('./topics', file), 'r').read())
            all_items += len(items)
            for item in items:
                if items[item] != []:
                    topics[item] = items[item]
                    with_topics += 1
            with open(file.replace('items_', 'topics_'), 'w') as outfile:
                json.dump(topics, outfile)
print(with_topics/all_items)
print(with_topics, all_items)

Всего топики есть у 0.5192952658431268 части элементов.

Элементов с топиками - 397466, без топиков - 367929. Всего - 765395.

Коллекций с топиками - 3206, без топиков - 1283. Всего - 4489.

Проверяем, нет ли среди элементов, из которых не извлечены топики, слова "topic" (вдруг они ещё как-то по-другому записываются?):

In [4]:
for root, dirs, files in os.walk('./topics'):
    for file in files:
        if file.startswith('errors'):
            print(file)
            errors = open('errors.json', 'r').read()
            n = re.findall('"topic"', errors)
            print(len(n))

errors_16ad5350-c52e-012f-aecf-58d385a7bc34.json
0
errors_5261fd50-c52e-012f-85ec-58d385a7bc34.json
0
errors_a301da20-c52e-012f-cc55-58d385a7bc34.json
0
errors_b50ab6f0-c52b-012f-5986-58d385a7bc34.json
0
errors_d6d29460-031a-0133-7ddc-58d385a7b928.json
0
errors_e5114e30-c52f-012f-993c-58d385a7bc34.json
0
errors_f7533140-3179-0134-f53a-00505686a51c.json
0


Проверили на семи корневых коллекциях, и хватит (запись таких элементов занимает время и память)

In [ ]:
http://api.repo.nypl.org/api/v1/collections/be0d4620-c5aa-012f-12ef-58d385a7bc34